In [1]:
import sqlite3
import pandas as pd
import sqlalchemy

In [2]:
engine=sqlalchemy.create_engine(r'sqlite:///C:\Users\56945\Downloads\database.sqlite')

In [5]:
# conexión realizada!
df=pd.read_sql('SELECT * FROM Player_Attributes LIMIT 5',engine)

In [8]:

df.columns

#promedio ponderado con las cualidades
#defensa
#medianamente alto, acceleration y resistencia ,defensa alta, fuerza, ambos taclke, cabezaso, ambos pases mas importante el corto, agression, intercepciom, marca, 2 ambidiestos y o 2izaueirdo y 2 derecho
#centro
# resistencia, agilidad, control pelota, velocidad, pase largo y corto, fuerza, dribiling, intercepciones, stamina, marking, salto, visión
# delantero
# wen atque, velocidad, pique, acelracion , reaccion, salto, control baol, cabezaso, posicionamiento, penalties, shot_power, agilidad, fuerza, pase corto y largo, 
#portero
#reacciomes, diving, marcaje, pie, mano (orden de importancia, solo esta cateroria esta ordewnada)
defensas=[altura,]

Index(['id', 'player_fifa_api_id', 'player_api_id', 'date', 'overall_rating',
       'potential', 'preferred_foot', 'attacking_work_rate',
       'defensive_work_rate', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes'],
      dtype='object')

# ocupar DB Browser for SQLite para hacer las queries de prueba y posteriormente conectarlas a un df con read_sql